### IMPORT REQUIRED PACKAGES

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
#import matplot.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

###  LOAD THE DATASETS

In [76]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [77]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [78]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [79]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

### DATA PREPROCESSING

In [80]:
# FILL THE NULL VALUES IN TRAIN DATA
for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    train_data[col].fillna(train_data[col].median(), inplace = True)
    test_data[col].fillna(test_data[col].median(), inplace = True)

In [81]:
# FILL THE NULL VALUES IN TEST DATA
for col in ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']:
    train_data[col].fillna(train_data[col].mode()[0], inplace = True)
    test_data[col].fillna(test_data[col].mode()[0], inplace = True)

In [82]:
train_data['CryoSleep'] = train_data['CryoSleep'].astype(bool)
test_data['CryoSleep'] = test_data['CryoSleep'].astype(bool)
train_data['VIP'] = train_data['VIP'].astype(bool)
test_data['VIP'] = test_data['VIP'].astype(bool)

### DROP UNWANTED COLUMN

In [83]:
train_data.drop(columns = ['Name'], inplace = True)
test_data.drop(columns = ['Name'], inplace = True)

### FEATURE ENGINEERING

In [84]:
train_data['Deck'] = train_data['Cabin'].apply(lambda x: x.split('/')[0])
train_data['Side'] = train_data['Cabin'].apply(lambda x: x.split('/')[2])

test_data['Deck'] = test_data['Cabin'].apply(lambda x: x.split('/')[0])
test_data['Side'] = test_data['Cabin'].apply(lambda x: x.split('/')[2])
# Drop the original Cabin column
train_data.drop(columns=['Cabin'], inplace=True)
test_data.drop(columns=['Cabin'], inplace=True)

### PANDAS'S GET DUMMIES

In [85]:
train_data = pd.get_dummies(train_data, columns =['HomePlanet', 'Destination', 'Deck', 'Side'], drop_first = True)
test_data = pd.get_dummies(test_data, columns =['HomePlanet', 'Destination', 'Deck', 'Side'], drop_first = True)

### MODEL BUILDING

In [86]:
X = train_data.drop(columns=['Transported', 'PassengerId'])
y = train_data['Transported']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_pred)}")

Validation Accuracy: 0.7826336975273146


### PREDICTION

In [87]:
# Predict on the test set
X_test = test_data.drop(columns=['PassengerId'])
test_data['Transported'] = model.predict(X_test)

# Prepare the submission file
submission = test_data[['PassengerId', 'Transported']]
submission.to_csv('submission.csv', index=False)
